In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import torch
from transformers import BertConfig
from fastformer import FastformerEncoder

c:\Users\madmo\miniconda3\envs\dl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
bert = BertModel.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')

In [6]:
from data_utils import *
import pickle 

build_news_id_map()
with open('data/MINDsmall_train/news_id_map.pkl', 'rb') as f:
    news_id_map = pickle.load(f)
build_user_id_map()
with open('data/MINDsmall_train/user_id_map.pkl', 'rb') as f:
    user_id_map = pickle.load(f)
build_news_token(news_id_map)
news_token = np.load('data/MINDsmall_train/news_token.npy')

Tokenizing: 100%|██████████| 51283/51283 [01:14<00:00, 689.70it/s]


In [9]:
bert_features = precompute_bert_features(bert, news_token, 'cuda')
torch.save(bert_features, 'bert_features.pt')

100%|██████████| 802/802 [00:31<00:00, 25.24it/s]


In [10]:
news_token

array([[ 101,  102,    0, ...,    0,    0,    0],
       [ 101, 1996, 9639, ...,    0,    0,    0],
       [ 101, 2753, 5409, ...,    0,    0,    0],
       ...,
       [ 101, 2358, 1012, ..., 2128, 4083,  102],
       [ 101, 2129, 1996, ...,    0,    0,    0],
       [ 101, 2190, 2998, ...,    0,    0,    0]],
      shape=(51283, 64), dtype=int32)

In [11]:
data_path = 'data/MINDsmall_train/'
edge_index = build_edge_index(data_path + 'behaviors.tsv', news_id_map=news_id_map)

100%|██████████| 153727/153727 [00:03<00:00, 48067.70it/s]


In [12]:
build_behaviors(data_path + 'behaviors.tsv', user_id_map, news_id_map, 'behaviors.pkl', 32)

156965it [00:03, 46292.97it/s]


In [13]:
with open('behaviors.pkl', 'rb') as f:
    behaviors_data = pickle.load(f)

In [14]:
from data import MINDDataset
from data_utils import collate_fn

In [15]:
mind_data = MINDDataset('behaviors.pkl')

In [16]:
from torch.utils.data import DataLoader

dataloader = DataLoader(mind_data, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [17]:
batch = next(iter(dataloader))

In [20]:
print(batch['user_idx'].shape)
print(batch['history'].shape)
print(batch['candidates'].shape)
print(batch['seed_nodes'].shape)

torch.Size([32])
torch.Size([32, 32])
torch.Size([32, 5])
torch.Size([690])
